In [81]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import itertools
import os
import re
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
zooms = [6,7,8]
modes = [4,9]
metadata = pd.read_csv("data/metadata.csv")
match = np.array(list(itertools.product(metadata['station_ID'], zooms)))
ratio = 0.75

In [88]:
#choice dataset
idxs = random.sample(range(match.shape[0]), int(match.shape[0]*ratio))
match_portion = match[idxs]
metadata.shape[0]*3, match_portion.shape

(20043, (15032, 2))

In [92]:
#check validity
set(np.unique(idxs, return_counts=True)[1]) == {1}

True

In [93]:
#files for learning
learning_files = []
for station_ID, zoom in match_portion:
    if station_ID < 10000:
        label = "true"
    else:
        label = "false"

    for mode in modes:
        file = f"station_{label}_{station_ID}_{zoom}_{mode}.png"
        learning_files.append(file)
learning_files_df = pd.DataFrame(learning_files)
learning_files_df.head()

,0
0,station_true_140_7_4.png
1,station_true_140_7_9.png
2,station_true_1804_8_4.png
3,station_true_1804_8_9.png
4,station_false_10027_8_4.png


In [113]:
#check validity
path = "./data/crop_map/"
is_true_files = True
for idx, lf in enumerate(learning_files_df.iloc[:,-1]):
    if not os.path.isfile(path + lf):
        print(idx)
        is_true_files = False

is_pair = True
pair = {}

for lf in learning_files_df.iloc[:,-1]:
    expression = r"station_(?P<label>\w+)_(?P<station_id>\d+)_(?P<zoom>\d+)_(?P<mode>\w+).png"
    regex = re.compile(expression)
    label,station_id,zoom,mode = regex.findall(lf)[0]
    if (station_id, zoom) not in pair.keys():
        pair[(station_id, zoom)] = [mode]
    else:
        pair[(station_id, zoom)].append(mode)
        
if not set([tuple(pv) for pv in pair.values()]) == {('4','9')}:
    is_pair = False

In [114]:
#save
if is_true_files and is_pair:
    learning_files_df.to_csv("learning_files.csv", header=False, index=False)